In [2]:
# this program is a test implementation of twitter sentiment analysis

# importing important libraries

import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import tweepy 
from tweepy import OAuthHandler 
# from textblob import TextBlob
from nltk.stem.porter import * 
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [8]:
# creating Twitter Client class

class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'lsgaJaqXwmEG53e6HuUsnrUDK'
        consumer_secret = 'vszcOZRFPokj1EVfg3HNTI0PCFUww6E2gllezodcu4XgkFVdvP'
        access_token = '2428900488-PEiE677RkaZeFdZdBF4HqFuTKiFkvmos3zYImmS'
        access_token_secret = 'yWaN0a0c9PAYf3a0XZl48SDeZmLLwGpOz9bJO5TNe5wd1'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
  
#     def get_tweet_sentiment(self, tweet): 
#         ''' 
#         Utility function to classify sentiment of passed tweet 
#         using textblob's sentiment method 
#         '''
#         # create TextBlob object of passed tweet text 
#         analysis = TextBlob(self.clean_tweet(tweet)) 
#         # set sentiment 
#         if analysis.sentiment.polarity < 0: 
#             return 1
#         else: 
#             return 0
  
    def get_tweets(self, query, count = 1000): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets
            
            maxId=0
            for i in range(100):
                fetched_tweets = self.api.search(q = query, count = count,max_id=maxId) 
                maxId=fetched_tweets.max_id

                # parsing tweets one by one 
                for tweet in fetched_tweets: 
                    # empty dictionary to store required params of a tweet 
                    parsed_tweet = {} 

                    # saving text of tweet 
                    parsed_tweet['text'] = tweet.text 
                    # saving sentiment of tweet 
#                     parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

                    # appending parsed tweet to tweets list 
                    if tweet.retweet_count > 0: 
                        # if tweet has retweets, ensure that it is appended only once 
                        if parsed_tweet not in tweets: 
                            tweets.append(parsed_tweet) 
                    else: 
                        tweets.append(parsed_tweet) 
                

            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))

     
    
    
api=TwitterClient()
tweets=api.get_tweets(query="trump")
total_data=pd.read_csv('empty.csv')
train=pd.read_csv('empty1.csv')
test=pd.read_csv('empty2.csv')
trainsize=int(len(tweets)*0.6)
for i in range(len(tweets)):
    if i<trainsize:
        train.loc[i]=[i+1,tweets[i]["sentiment"],tweets[i]["text"]]
    else:
        test.loc[i]=[i+1,tweets[i]["sentiment"],tweets[i]["text"]]
    total_data.loc[i]=[i+1,tweets[i]["sentiment"],tweets[i]["text"]]
    i=i+1
    
total_data.head()

# loading training and test data

#train  = pd.read_csv('Dataset/train_tweets.csv')
#test = pd.read_csv('Dataset/test_tweets.csv')

#combining training and and test data for preprocessing 
#total_data = train.append(test, ignore_index=True)
print(len(total_data))
print(len(train))
print(len(test))

1258781334819889151
1258781324157968384
1258781314045685759
1258781302200958977
1258781292600201215
1258781281602736130
1258781270340820991
1258781260534562815
1258781250376040448
1258781241798729730
1258781232847978496
1258781222412513279
1258781213357158401
1258781204028981248
1258781194801553407
1258781184701665282
1258781175511801855
1258781165986754561
1258781155978907648
1258781145673629696
1258781135401816064
1258781124609875970
1258781115004813313
1258781104938430464
1258781096394788863
1258781087167365121
1258781077713321983
1258781068183781375
1258781059275202559
1258781049045344256
1258781039247323135
1258781029344706559
1258781019060277247
1258781008817803263
1258780998260723711
1258780987821088767
1258780978392313855
1258780968648876033
1258780958322569217
1258780948260425727
1258780938772742146
1258780928199069697
1258780917281087487
1258780907030355969
1258780896100048895
1258780886302101503
1258780877506707456
1258780866689601538
1258780856832925695
1258780846514937855


KeyboardInterrupt: 

In [49]:
total_data.head()

,id,label,tweet
0,1,0,RT @pdmcleod: The tl;dr of the 300-page is thi...
1,2,0,RT @Catmandu50: Sedition: Pelosi Travels to Ma...
2,3,0,@AZachParkinson Trump kept the real racist out
3,4,1,"RT @ddale8: No, Trump didn’t predict Brexit th..."
4,5,0,"RT @TheWayWithAnoa: Dear White Moderates,\n\nH..."


In [50]:
#function to remove @word pattern from the tweets as they do not add any value

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [51]:
# removing @username patterns from tweets using remove_pattern function

print('\n\nRemoving  Twitter Handles \n\n')
total_data['tidy_tweet'] = np.vectorize(remove_pattern)(total_data['tweet'], "@[\w]*")
total_data.head()



Removing  Twitter Handles 




,id,label,tweet,tidy_tweet
0,1,0,RT @pdmcleod: The tl;dr of the 300-page is thi...,RT : The tl;dr of the 300-page is this line: “...
1,2,0,RT @Catmandu50: Sedition: Pelosi Travels to Ma...,RT : Sedition: Pelosi Travels to Madrid for UN...
2,3,0,@AZachParkinson Trump kept the real racist out,Trump kept the real racist out
3,4,1,"RT @ddale8: No, Trump didn’t predict Brexit th...","RT : No, Trump didn’t predict Brexit the day b..."
4,5,0,"RT @TheWayWithAnoa: Dear White Moderates,\n\nH...","RT : Dear White Moderates,\n\nHow many times d..."


In [52]:
# words with small lenght i.e., words having length smaller than 3 hardly hold any sentiment
# hence it is better to remove such words

print('\n\nRemoving Short Words\n\n')
total_data['tidy_tweet'] = total_data['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
total_data.head()



Removing Short Words




,id,label,tweet,tidy_tweet
0,1,0,RT @pdmcleod: The tl;dr of the 300-page is thi...,tl;dr 300-page this line: “President Trump com...
1,2,0,RT @Catmandu50: Sedition: Pelosi Travels to Ma...,Sedition: Pelosi Travels Madrid Climate Meetin...
2,3,0,@AZachParkinson Trump kept the real racist out,Trump kept real racist
3,4,1,"RT @ddale8: No, Trump didn’t predict Brexit th...",Trump didn’t predict Brexit before. didn’t per...
4,5,0,"RT @TheWayWithAnoa: Dear White Moderates,\n\nH...","Dear White Moderates, many times do... Black, ..."


In [53]:
# separating each word as a token

print('\n\nTweet Tokenization\n\n')
tokenized_tweet = total_data['tidy_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()



Tweet Tokenization




0    [tl;dr, 300-page, this, line:, “President, Tru...
1    [Sedition:, Pelosi, Travels, Madrid, Climate, ...
2                          [Trump, kept, real, racist]
3    [Trump, didn’t, predict, Brexit, before., didn...
4    [Dear, White, Moderates,, many, times, do..., ...
Name: tidy_tweet, dtype: object

In [54]:
# stemming words i.e, words are play,playing,played are treated similarly

print('\n\nStemming\n\n')
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])
tokenized_tweet.head()



Stemming




0    [tl;dr, 300-page, thi, line:, “presid, trump, ...
1    [sedition:, pelosi, travel, madrid, climat, me...
2                          [trump, kept, real, racist]
3    [trump, didn’t, predict, brexit, before., didn...
4    [dear, white, moderates,, mani, time, do..., b...
Name: tidy_tweet, dtype: object

In [55]:
#stiching these tokens together

for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
total_data['tidy_tweet'] = tokenized_tweet
total_data.head()

,id,label,tweet,tidy_tweet
0,1,0,RT @pdmcleod: The tl;dr of the 300-page is thi...,tl;dr 300-page thi line: “presid trump comprom...
1,2,0,RT @Catmandu50: Sedition: Pelosi Travels to Ma...,"sedition: pelosi travel madrid climat meeting,..."
2,3,0,@AZachParkinson Trump kept the real racist out,trump kept real racist
3,4,1,"RT @ddale8: No, Trump didn’t predict Brexit th...",trump didn’t predict brexit before. didn’t per...
4,5,0,"RT @TheWayWithAnoa: Dear White Moderates,\n\nH...","dear white moderates, mani time do... black, l..."


In [56]:
# checking out the features of the total_data dataframe

for col in total_data.columns:
	print(col)

id
label
tweet
tidy_tweet


In [57]:
# converting tidy_tweets column to numerical value using bag of words algorithm

'''
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90,min_df=2,max_features=1000, stop_words='english')

# bag-of-words feature matrix

bow = bow_vectorizer.fit_transform(total_data['tidy_tweet'])

bow=bow.toarray()
print(bow.shape)

# get an idea of bow array as it is difficult to visualise in normal form
#hence converting it to a numpy array
'''
# implementing bag of words

word2count = {}
wordlist=[]
for data in total_data['tidy_tweet'].values: 
    words = nltk.word_tokenize(data)
    wordlist.append(words)
    for word in words: 
        if word not in word2count.keys(): 
            word2count[word] = 1
        else: 
            word2count[word] += 1

In [58]:
import heapq 
freq_words = heapq.nlargest(1000, word2count, key=word2count.get)

bow = [] 
for data,lword in zip(total_data['tidy_tweet'].values,wordlist): 
    vector = [] 
    for word in freq_words: 
        if word in lword: 
            vector.append(1) 
        else: 
            vector.append(0) 
    bow.append(vector) 
bow = np.asarray(bow)       

In [59]:
print(bow.shape)

(4399, 1000)


In [60]:
# import classes for logistic regression

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [61]:
# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(bow, total_data['label'],  test_size=0.2)

In [62]:
print(xtrain_bow.shape)
print(xvalid_bow.shape)
print(ytrain.shape)
print(yvalid.shape)

(3519, 1000)
(880, 1000)
(3519,)
(880,)


In [63]:
lreg=LogisticRegression()
lreg.fit(xtrain_bow,list(ytrain))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [64]:
predict=lreg.predict(xvalid_bow)
predict.shape

(880,)

In [65]:
predict=lreg.predict(xvalid_bow)
print(np.asarray(yvalid))
print(predict)

[0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0
 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0
 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0
 0 0 1 0 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1
 1 0 0 0 1 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1
 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0
 1 1 0 0 0 0 0 0 1 0 0 0 

In [66]:
prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

print(f1_score(list(yvalid), prediction_int))

0.5776699029126214
